# Test Gridsearch
with different kernels and different configurations

# Setup
Uncomment the next cell if STree is not already installed

In [1]:
#
# Google Colab setup
#
#!pip install git+https://github.com/doctorado-ml/stree

In [2]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV, train_test_split
from stree import Stree

In [3]:
import os
if not os.path.isfile('data/creditcard.csv'):
    !wget --no-check-certificate --content-disposition http://nube.jccm.es/index.php/s/Zs7SYtZQJ3RQ2H2/download
    !tar xzf creditcard.tgz

In [4]:
random_state=1

def load_creditcard(n_examples=0):
    import pandas as pd
    import numpy as np
    import random
    df = pd.read_csv('data/creditcard.csv')
    print("Fraud: {0:.3f}% {1}".format(df.Class[df.Class == 1].count()*100/df.shape[0], df.Class[df.Class == 1].count()))
    print("Valid: {0:.3f}% {1}".format(df.Class[df.Class == 0].count()*100/df.shape[0], df.Class[df.Class == 0].count()))
    y = df.Class
    X = df.drop(['Class', 'Time', 'Amount'], axis=1).values
    if n_examples > 0:
        # Take first n_examples samples
        X = X[:n_examples, :]
        y = y[:n_examples, :]
    else:
        # Take all the positive samples with a number of random negatives
        if n_examples < 0:
            Xt = X[(y == 1).ravel()]
            yt = y[(y == 1).ravel()]
            indices = random.sample(range(X.shape[0]), -1 * n_examples)
            X = np.append(Xt, X[indices], axis=0)
            y = np.append(yt, y[indices], axis=0)
    print("X.shape", X.shape, " y.shape", y.shape)
    print("Fraud: {0:.3f}% {1}".format(len(y[y == 1])*100/X.shape[0], len(y[y == 1])))
    print("Valid: {0:.3f}% {1}".format(len(y[y == 0]) * 100 / X.shape[0], len(y[y == 0])))
    Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=random_state, stratify=y)
    return Xtrain, Xtest, ytrain, ytest

data = load_creditcard(-1000) # Take all true samples + 1000 of the others
# data = load_creditcard(5000)  # Take the first 5000 samples
# data = load_creditcard(0) # Take all the samples

Xtrain = data[0]
Xtest = data[1]
ytrain = data[2]
ytest = data[3]

Fraud: 0.173% 492
Valid: 99.827% 284315
X.shape (1492, 28)  y.shape (1492,)
Fraud: 32.976% 492
Valid: 67.024% 1000


# Tests

In [5]:
parameters = {
    'base_estimator': [Stree()],
    'n_estimators': [10, 25],
    'learning_rate': [.5, 1],
    'base_estimator__tol': [.1,  1e-02],
    'base_estimator__max_depth': [3, 5],
    'base_estimator__C': [7, 55],
    'base_estimator__kernel': ['linear', 'poly', 'rbf']
}

In [6]:
Stree().get_params()

{'C': 1.0,
 'criterion': 'gini',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'linear',
 'max_depth': None,
 'max_features': None,
 'max_iter': 1000,
 'min_samples_split': 0,
 'random_state': None,
 'split_criteria': 'max_samples',
 'splitter': 'random',
 'tol': 0.0001}

In [7]:
random_state=2020
clf = AdaBoostClassifier(random_state=random_state, algorithm="SAMME")
grid = GridSearchCV(clf, parameters, verbose=10, n_jobs=-1, return_train_score=True)
grid.fit(Xtrain, ytrain)

Fitting 5 folds for each of 96 candidates, totalling 480 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    6.6s
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:    8.1s
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:    9.4s
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:   10.1s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:   11.1s
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed:   12.3s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   13.6s
[Parallel(n_jobs=-1)]: Done 165 tasks      | elapsed:   

GridSearchCV(estimator=AdaBoostClassifier(algorithm='SAMME', random_state=2020),
             n_jobs=-1,
             param_grid={'base_estimator': [Stree(C=55, max_depth=3, tol=0.01)],
                         'base_estimator__C': [7, 55],
                         'base_estimator__kernel': ['linear', 'poly', 'rbf'],
                         'base_estimator__max_depth': [3, 5],
                         'base_estimator__tol': [0.1, 0.01],
                         'learning_rate': [0.5, 1], 'n_estimators': [10, 25]},
             return_train_score=True, verbose=10)

In [8]:
print("Best estimator: ", grid.best_estimator_)
print("Best hyperparameters: ", grid.best_params_)
print("Best accuracy: ", grid.best_score_)

Best estimator:  AdaBoostClassifier(algorithm='SAMME',
                   base_estimator=Stree(C=55, max_depth=3, tol=0.01),
                   learning_rate=0.5, n_estimators=25, random_state=2020)
Best hyperparameters:  {'base_estimator': Stree(C=55, max_depth=3, tol=0.01), 'base_estimator__C': 55, 'base_estimator__kernel': 'linear', 'base_estimator__max_depth': 3, 'base_estimator__tol': 0.01, 'learning_rate': 0.5, 'n_estimators': 25}
Best accuracy:  0.9559440559440558
